# Predict the price of a product based on features like you would with a car price. 

- Summary: 


- Hints:
    - First create a super simple framework for prediction include:
        - include kfolds, scoring, data input etc. and do this with 1 feature to get started and keep focused on end goal. 
    - when cleansing the data, create simple processes that can be reused when new data sets for testing and using of the system are predicted on. 
        - Examples include when we standardize data in the original fit we need to save off the mean and variance used and apply to the new column when it arrives in the predict. 
        - Another example could be when a categorical column arrives for prediction with a new unaccounted for value. say I predicted on A, B, C, but D arrives in the prediction test set. How do we handle this? 
    - any cleansing of the data should be reused on test data. So we want to avoid manual cleaning except for validating the process first. 
    

- Side notes:
    - when using linear model regression use StatsModels rather than sklearn because StatsModel has a little more info that we want to use.
    
    

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from sklearn.model_selection import KFold

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 10000)

/Users/michaelwoline/anaconda2/envs/python36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_dictionary = pd.read_excel('Data Dictionary.xlsx')

In [3]:
df_dictionary.info()
df_dictionary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 3 columns):
Variable       53 non-null object
Description    53 non-null object
Unnamed: 2     1 non-null object
dtypes: object(3)
memory usage: 1.3+ KB


,Variable,Description,Unnamed: 2
0,SalesID,unique identifier of a particular sale of a ...,NaN
1,MachineID,identifier for a particular machine; machin...,NaN
2,ModelID,identifier for a unique machine model (i.e. ...,NaN
3,datasource,source of the sale record; some sources are...,NaN
4,auctioneerID,"identifier of a particular auctioneer, i.e. ...",NaN
5,YearMade,year of manufacturer of the Machine,NaN
6,MachineHoursCurrentMeter,current usage of the machine in hours at tim...,NaN
7,UsageBand,"value (low, medium, high) calculated compari...",NaN
8,Saledate,time of sale,NaN
9,Saleprice,cost of sale in USD,NaN


In [4]:
key_fields_list = ['']

In [5]:
data_df = pd.read_csv('Train.csv')

/Users/michaelwoline/anaconda2/envs/python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       4

In [7]:
print(len(set(data_df['ProductGroup'])))
print(set(data_df['ProductGroup']))
# data_df.iloc[0:10].as_matrix()
data_df.head(1000)


6
{'SSL', 'MG', 'WL', 'TEX', 'BL', 'TTT'}


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,12/18/2008 0:00,310G,310,G,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Arizona,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Extended,Powershuttle,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,8/26/2004 0:00,790ELC,790,E,NaN,LC,Large / Medium,"Hydraulic Excavator, Track - 21.0 to 24.0 Metr...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,11/17/2005 0:00,416D,416,D,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Illinois,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,Reversible,No,Standard,Standard,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,8/27/2009 0:00,430HAG,430,HAG,NaN,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Texas,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,Manual,NaN,NaN,NaN,Rubber,None or Unspecified,None or Unspecified,None or Unspecif

In [8]:
print(len(set(data_df['MachineID'])))
print(len(set(data_df['ModelID'])))
np.sum(data_df['ModelID'] ==np.nan)



341027
5218


0

### Data Cleansing and transformation here:


In [9]:
# Clean data added to new processing dataframe. called clean_data_df.

# Encode ProductGroup data
# When test data arrives, it will need to be processed and checkded if all categories exist first, and if not
# then probably use another prefit model. 
clean_data_df = pd.concat([data_df['SalePrice'], pd.get_dummies(data_df['ProductGroup'], prefix='ProductGroup_')], axis=1)

clean_data_df.head(1000)


,SalePrice,ProductGroup__BL,ProductGroup__MG,ProductGroup__SSL,ProductGroup__TEX,ProductGroup__TTT,ProductGroup__WL
0,66000,0,0,0,0,0,1
1,57000,0,0,0,0,0,1
2,10000,0,0,1,0,0,0
3,38500,0,0,0,1,0,0
4,11000,0,0,1,0,0,0
5,26500,1,0,0,0,0,0
6,21000,0,0,0,1,0,0
7,27000,1,0,0,0,0,0
8,21500,0,0,0,1,0,0
9,65000,0,0,0,0,0,1


In [10]:
y = clean_data_df['SalePrice'].as_matrix()
X = clean_data_df[['ProductGroup__BL','ProductGroup__MG','ProductGroup__SSL', 'ProductGroup__TEX', 'ProductGroup__TTT']].as_matrix()

type(X)
print(y.shape)
print(X.shape)

(401125,)
(401125, 5)


In [11]:
n_splits = 3
kf = KFold(n_splits, random_state=0, shuffle=True)

r_squared_score = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = sm.OLS(y_train, X_train).fit()
    predictions = model.predict(X_test)

    # score the model
#     r_squared_score += np.sum((y_test[0] - predictions[0])**2)/predictions.shape[0]
    r_squared_score += (np.corrcoef(y_test, predictions)**2)[0,1]
    print (r_squared_score)
#     print(model.summary())
print("r_squared out of sample average score ", r_squared_score/n_splits)
model.summary()


0.0322225111165
0.0627095866881
0.0936044722409
r_squared out of sample average score  0.031201490747


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                 6.545e+04
Date:                Thu, 01 Feb 2018   Prob (F-statistic):               0.00
Time:                        17:24:13   Log-Likelihood:            -3.0974e+06
No. Observations:              267417   AIC:                         6.195e+06
Df Residuals:                  267412   BIC:                         6.195e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2.094e+04    112.557    186.051      0.000    2.07e+04    2.12e+04
x2          4.735e+04    199.342    237.557      0.000     4.7e+04    4.77e+04
x3          1.055e+04    152.583     69.154      0.000    1.03e+04    1.09e+04
x4          3.569e+04     99.823    357.501      0.000    3.55e+04    3.59e+04
x5          3.612e+04    112.224    321.891      0.000    3.59e+04    3.63e+04
==============================================================================
Omnibus:                    43168.498   Durbin-Watson:                   0.548
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70141.190
Skew:                           1.105   Prob(JB):                         0.00
Kurtosis:                       4.189   Cond. No.                         2.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
print(predictions.shape)
print(y_test.shape)


# temp = sum((y_test - predictions)**2)/y_test.shape[0]
# temp2 = np.corrcoef(y_test, predictions)**2
predictions = model.predict(X_train)
print((np.corrcoef( predictions, y_train)**2)[0,1])
print((np.corrcoef(y_train, predictions)))
# print(coefficient_of_determination(X_train, predictions))

(133708,)
(133708,)
0.0313191769834
[[ 1.          0.17697225]
 [ 0.17697225  1.        ]]


In [13]:
def coefficient_of_determination(ys_orig,ys_line):
    y_mean_line = [np.mean(ys_orig) for y in ys_orig]
    squared_error_regr = squared_error(ys_orig, ys_line)
    squared_error_y_mean = squared_error(ys_orig, y_mean_line)
    return (1 - (squared_error_regr/squared_error_y_mean))

def squared_error(ys_orig,ys_line):
    return sum((ys_line - ys_orig) * (ys_line - ys_orig))

- R^2 is proportion of variation explained by the model. 
- RSS residual sum of squares. this is the difference between y_hat and actual y
- total sum of squares. 

- R^2 = 1- RSS/TSS
- Capital R represents the model
- lower r represents a statistic
- R = r(y, y_hat)

- total variation across y global from 0 to max is called TSS (total sum of squares)

- adjusted R^2 is an attempt to penalize for model complexity that gives an indicator of overall score. Old school, not used very often, but can by a general indicator if very different from R^2 that there could be an issue here. 
    - Out K-folds does better. 

- F statistic - when high, if anything that is put in the model I can't tell if there is anything that tells me clearly what the price should be.
- Prob (F-statistic) (P-statistic) for the overall model. When less then .05 then the model is performing well and reject the null hypothosis that the model is performing better than chance. statistically. 
    - if > .05 then we can't reject the null hypothesis. null hypothesis says I can guess by rolling a dice what the price should be. There is not enough prediction in the overall model. 
    - This is the first stop and check to see if I can proceed with my fitting of the model. 
        - if there is a score less than .05, then there is at least one feature that is predicting well, and go ahead and keep going.
            - from here I can look at individual features to see which one had a good p-score that says reject or fail to reject some features. 


- Cond Number - at the bottom of the OLS Results. this tells us how difficult it was to do matrix inversion. 
    - big numbers are in the 1000's and they are bad. 
    - if there is a big number it tells us there is either colinearity or our features exist on different scales and need to be standardized.  
    


In [74]:
model.summary()

numpy.ndarray